In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# 네이버 뉴스 데이터 가져오기
1. 네이버 오픈 API로 기사 URL 크롤링
2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [ ]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 3

client_id = "h6NTRMHO6mjOXhu18YsU"
client_secret = "EPOzKae29u"
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
  url = "https://openapi.naver.com/v1/search/news?query=" + encText + '&start=' + str(idx*10 + 1) # json 결과
  # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과

  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()

  if(rescode==200):
    result = requests.get(response.geturl(),
                          headers={'X-Naver-Client-Id':client_id,
                                  'X-Naver-Client-Secret':client_secret})
    news_data.append(result.json())
  else:
      print("Error Code:" + rescode)

## 네이버 open api 통해 가져온 데이터 확인

In [ ]:
print(news_data[0]['items'][0]['link'])

http://www.lecturernews.com/news/articleView.html?idxno=89644


## 가져온 url이 네이버 뉴스인지 확인

In [ ]:
naver_news_link = []

for page in news_data:

  page_news_link = []

  for item in page['items']:
    link = item['link']
    if 'naver' in link:
      page_news_link.append(link)

  naver_news_link.append(page_news_link)

for page in naver_news_link:
  for link in page:
    print(link)

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=081&aid=0003251300
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004707412
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010998604
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=648&aid=0000006347
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010996248
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=008&aid=0004706786
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005897791
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004706534
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002247264
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=421&aid=0005895811


In [ ]:
!pip install selenium

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import time

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome('chromedriver',options=options)

In [ ]:
naver_news_title = []
naver_news_content = []

for n in tqdm(range(len(naver_news_link))):
  # print(n)
  news_page_title = []
  news_page_content = []

  for idx in tqdm(range(len(naver_news_link[n]))):

  ##### URL 접속 #####
    try:
      driver.get(naver_news_link[n][idx])
      print(naver_news_link[n][idx])

    except:
      print('Timeout')
      continue

    try:
      response = driver.page_source

    except UnexpectedAlertPresentException:
      driver.switch_to_alert().accept()
      print('게시글이 삭제된 경우입니다.')
      continue

    soup = BeautifulSoup(response, 'html.parser')

    ##### 뉴스 제목 #####
    title = None

    try:
      item = soup.find('div', class_='article_info')
      title = item.find('h3', class_='tts_head').get_text()

    except:
      title = 'OUTLINK'

    news_page_title.append(title)

    ##### 뉴스 본문 #####
    doc = None
    text = ""

    data = soup.find_all('div', {'class':'_article_body_contents'})
    if data:
      for item in data:
        text = text + str(item.find_all(text=True)).strip()
        text = ast.literal_eval(text)
        doc = ' '.join(text)
    else:
      doc = 'OUTLINK'

    news_page_content.append(doc.replace('\n', ' '))
  
  naver_news_title.append(news_page_title)
  naver_news_content.append(news_page_content)

  time.sleep(2)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=081&aid=0003251300
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004707412
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010998604


  0%|          | 0/3 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=648&aid=0000006347
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010996248
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=008&aid=0004706786


  0%|          | 0/4 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005897791
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004706534
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002247264
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=421&aid=0005895811


## 크롤링 결과 확인

In [ ]:
print(naver_news_title[0])

['코로나 이후 아동 성착취물 246% 폭증… 줄어든 강력범죄, 온라인으로 자리 옮겨', "청담러닝+씨엠에스에듀 '교육계 흔들 비트코인' 베팅하는 펀드들", '[교육소식]공주대 보건의료 데이터마이닝 기초과정 등']


In [ ]:
print(naver_news_content[0])

['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    <데이터로 뉴스 읽기> 범죄 양상까지 바꾼 코로나 2014~2020 년 범죄 데이터 분석 사이버범죄  74 % 직거래 등 피해 성인 성착취물 연평균  12 %P 감소 사이버음란물의  54 % 아동 대상 30 대 남성 A씨와  20 대 남성 B씨는 코로나 19 로 캠핑 수요가 늘어나자  2020 년 7월부터 지난해 3월까지 온라인 중고거래 사이트에 고가의 캠핑 용품 등을 싸게 판매한다는 글을 올리고 구입자  130 명에게서 거래대금 1억  5000 만원을 가로챘다. 인터넷 등에 다른 사람들이 올린 장비 사진을 도용해 자신들의 이름과 날짜 등을 합성하는 수법을 썼다. 이후 구입자들에게 택배 송장번호를 보낸 뒤 바로 택배를 취소하거나 상자에 돌을 넣어 보내는 방식으로 속였다. 전형적인 ‘사이버사기’다. ●코로나 이후 사이버범죄  22.8 %P 늘어 코로나 19 로 인한 사회적 거리두기 기간 동안 폭력, 절도 등 강력범죄는 줄고 사이버사기, 사이버금융범죄 등 온라인상 범죄는 증가한 것으로 나타났다. 서울신문이  13 일 경찰청이 공공데이터포털에 공개한  2014 년부터  2020 년까지 연도별 사이버범죄 현황과 전국 경찰서별 강력범죄 발생 현황을 프로그래밍 언어 파이썬을 이용해 분석한 결과 4대 강력범죄(살인·강도·절도·폭력)는  2014 년  55 만  8012 건에서  2020 년  44 만  5845 건으로 약  20 % 감소했으나 사이버범죄는  2014 년  10 만  9979 건에서  2020 년  23 만  4095 건으로 2배 이상 늘었다. 특히 7년간 연평균  10 % 포인트였던 사이버범죄 증가율이  2020 년엔 코로나 19  발생 이전인  2019 년 대비  22.8 % 포인트로 크게 증가했다. 다만 강력범죄와 사이버범죄를 합산한 전체 범죄 발생 건수는  2014

In [ ]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

3
3


## 크롤링 결과 저장

In [ ]:
with open('/content/drive/MyDrive/DL/AtoZ/crawling/naver_news_title.pk', 'wb') as f:
  pickle.dump(naver_news_title, f)

with open('/content/drive/MyDrive/DL/AtoZ/crawling/naver_news_content.pk', 'wb') as f:
  pickle.dump(naver_news_content, f)